# Adding Back the Tides - CIOPS
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

#### unlike what we did with SSC in this file make all add back tides days seperate to reduce space requirement
#### just doing 2017 for now (only full CIOPS year without eddy isue in SSC)
plus 30 days on the end for running purposes

In [2]:
startday = [dt.datetime(2018,1,28)+dt.timedelta(days=i) for i in range(int(14*7))]
print(startday[-1])

2018-05-05 00:00:00


In [3]:
folders = [dt.datetime(2018,1,28)+dt.timedelta(days=7*(i+1)) for i in range(int(14))]
# print(folders[-1])
folders = np.repeat(folders,7)
print(folders[0])

2018-02-04 00:00:00


In [4]:
# start = dt.datetime(2015,11,22) #start day of your run

#dates for each run
# numdays = 2 #15 for all except the last run (7) 
# date_list = [start + dt.timedelta(days=x) for x in range(numdays)]
# folderday = [start+ dt.timedelta(days=7),start+ dt.timedelta(days=7*2)]#,start+ dt.timedelta(days=7*3)]
# folderday = np.repeat(folderday,7)
# folderday = folderday[:-6]
date_list_daily = [startday[0],startday[1]]
folderday_daily = [folders[0], folders[1]]

# for whatever reason the naming it raelly really weird and the daily and the hourly files names for the SAME DAY aren't\
# on the same day.. must take hourly files a day ahead woooooooooooo
date_list_hourly = [startday[1],startday[2]]
folderday_hourly = [folders[1], folders[2]]

In [5]:
"{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[0], date_list_hourly[0], date_list_hourly[0])

'2018020400/BC12_1h_grid_U_2D_20180129_20180129.nc'

In [6]:
path = Path("/ocean/mdunphy/CIOPSW-BC12/")

drop_vars = (
    "sbu", "tauuo", "time_counter_bounds", "time_instant_bounds", "uos", "time_instant",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[i], date_list_hourly[i], date_list_hourly[i]))) for i in range(len(date_list_hourly))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
ut_h = mydata['ubar']

In [7]:
drop_vars = (
    "sbv", "tauvo", "time_counter_bounds", "time_instant_bounds", "vos", "time_instant",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_V_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_hourly[i], date_list_hourly[i], date_list_hourly[i]))) for i in range(len(date_list_hourly))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
vt_h = mydata['vbar']

In [8]:
# 2D hourly outputs are alraedy in bartropic form so you dont need to do any conversion!

In [12]:
# Now get the required data from the daily files
drop_vars = (
    "depthu_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_U_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_daily[i], date_list_daily[i], date_list_daily[i]))) for i in range(len(date_list_daily))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['uo']

drop_vars = (
    "depthv_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_V_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_daily[i], date_list_daily[i], date_list_daily[i]))) for i in range(len(date_list_daily))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vo']

In [13]:
u_d = u_d.where(u_d != 0)
u_d

<xarray.DataArray 'uo' (time_counter: 2, depthu: 75, y: 1020, x: 714)>
dask.array<where, shape=(2, 75, 1020, 714), dtype=float32, chunksize=(1, 75, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 2018-01-29 2018-01-30
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_x_velocity
    long_name:           Sea Water X Velocity
    units:               m/s
    online_operation:    instant
    interval_operation:  1 d
    interval_write:      1 d
    cell_methods:        time: point

In [14]:
# i think since we want to do this with variable ssd we want to make the u_d be "hourly" from the get-go (before calcs that involve e3t)
offset = dt.timedelta(hours=1) # daily avg timestamp as next day time 0, offset by -23 hours to line up with hourly data
u_d = u_d.resample(time_counter="1H", loffset=offset).interpolate("linear")
u_d

<xarray.DataArray 'uo' (time_counter: 25, depthu: 75, y: 1020, x: 714)>
dask.array<transpose, shape=(25, 75, 1020, 714), dtype=float32, chunksize=(25, 75, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 2018-01-29T01:00:00 ... 2018-...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_x_velocity
    long_name:           Sea Water X Velocity
    units:               m/s
    online_operation:    instant
    interval_operation:  1 d
    interval_write:      1 d
    cell_methods:        time: point

In [15]:
# we WILL need to do some conversions here, so get e3t from the mesh_mask file
mydata = xr.open_dataset("/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc")
e3t = mydata['e3t_0'][0,:,:,:]

In [16]:
# now bring in ssh for the same days
drop_vars = (
    "mldkz5", 'mldr10_1', "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 'sbs', 'sbt', 'sos', 'ssh_ib', 'ssh_tide', 'tos'
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_T_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday_daily[i], date_list_daily[i], date_list_daily[i]))) for i in range(len(date_list_daily))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
eta = mydata['zos']

In [17]:
H = np.sum(e3t,axis=0)

In [18]:
e3t_ssh = e3t+((eta*e3t)/H)

In [19]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t_ssh.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5 #check to make sure this worked
e3u = e3u.rename({'z': 'depthu'})

e3t_yshift = e3t_ssh.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'z': 'depthv'})

In [20]:
mydata = xr.open_dataset("/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc")
xmesh_u = mydata.rename({'z': 'depthu'})
xmesh_v = mydata.rename({'z': 'depthv'})
umask = xmesh_u.vmask[0,:,:,:]*xmesh_u.umask[0,:,:,:]
vmask = xmesh_v.vmask[0,:,:,:]*xmesh_v.umask[0,:,:,:]

In [23]:
#calcuate bartropic component
ut_d = (u_d*e3u*umask).sum(dim='depthu')/(e3u*umask).sum(dim='depthu')
ut_d

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.DataArray (time_counter: 24, y: 1020, x: 714)>
dask.array<truediv, shape=(24, 1020, 714), dtype=float64, chunksize=(24, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2018-01-29T01:00:00 ... 2018-...
Dimensions without coordinates: y, x

In [24]:
# subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?
uc_d

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.DataArray (time_counter: 24, depthu: 75, y: 1020, x: 714)>
dask.array<sub, shape=(24, 75, 1020, 714), dtype=float64, chunksize=(24, 75, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2018-01-29T01:00:00 ... 2018-...
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

interpolate + resample uc_d to get it in an hourly format

In [23]:
# uc_d[0,:,646,267].values

In [24]:
# offset = dt.timedelta(hours=1) # daily avg timestamp as next day time 0, offset by -23 hours to line up with hourly data
# uc_h_interp = uc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")
#instead of taking 12hours off each side like when done with SSC, this method takes off 24hours on the end!
# , loffset=offset

In [92]:
# uc_h_interp

In [27]:
# uc_h_interp[0,0,646,267].values

In [25]:
u_new = ut_h  + uc_d

In [26]:
u_new

<xarray.DataArray (time_counter: 24, y: 1020, x: 714, depthu: 75)>
dask.array<add, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2018-01-29T01:00:00 ... 2018-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [41]:
# u_new = u_new.isel(time_counter = np.arange(0,24,1))

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [27]:
u_new = u_new.rename('vozocrtx')

In [28]:
u_new

<xarray.DataArray 'vozocrtx' (time_counter: 24, y: 1020, x: 714, depthu: 75)>
dask.array<add, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2018-01-29T01:00:00 ... 2018-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [44]:
# u_new[0,646,267,0].values

In [45]:
# xmesh = xr.open_dataset('/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc')
# xmesh_u = xmesh.rename({'z': 'depthu'})
# xmesh_v = xmesh.rename({'z': 'depthv'})

u_new = u_new*umask*vmask

In [46]:
u_new

<xarray.DataArray (time_counter: 24, y: 1020, x: 714, depthu: 75)>
dask.array<mul, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * depthu        (depthu) float64 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 2016-12-26T01:00:00 ... 2016-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
Dimensions without coordinates: y, x

In [49]:
encoding={
          "vozocrtx": {"zlib": True, "complevel": 4, "_FillValue": np.nan}
}

In [32]:
# encoding = {
#     "depthu": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#             "depthu": 75,
#         }
#     },
#     "vozocrtx": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#             "time_counter": 1, "y": 1020, "x": 714, "depthu": 75,
#         }
#     }
# }

# encoding = {
#     "depthu": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": [75],
#     },
#     "time_counter": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": [1],
#         "units": "hours since 1970-01-01 00:00:00"
#     },
#     "nav_lat": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#         "y": 1020, "x": 714,
#         }
#     },
#     "nav_lon": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#         "y": 1020, "x": 714,
#         }
#     },
#     "vozocrtx": {
#         "zlib": True, "complevel": 4,
#         "chunksizes": {
#         "time_counter": 1, "y": 1020, "x": 714, "depthu": 75,
#         }
#     }
# }

In [52]:
path = "/ocean/rbeutel/data/"
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_fill.nc'.format(date_list_hourly[0],date_list_hourly[0]), encoding=encoding)
print('u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list_hourly[0],date_list_hourly[0]))

u_new_23Nov15_23Nov15.nc


In [27]:
#calcuate bartropic component
vt_d = (v_d*e3v[0,:,:,:]).sum(dim='depthv')/e3v[0,:,:,:].sum(dim='depthv')

In [29]:
#subtract from v to get baroclinic component
vc_d = v_d-vt_d 

In [30]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")

In [31]:
v_new = vt_h  + vc_h_interp

In [32]:
v_new = v_new.isel(time_counter = np.arange(0,24,1))
v_new = v_new.rename('vomecrty')

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [33]:
v_new

<xarray.DataArray 'vomecrty' (time_counter: 24, y: 1020, x: 714, depthv: 75)>
dask.array<getitem, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2016-04-01T01:00:00 ... 2016-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
  * depthv        (depthv) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))